# Phase 2, Step 4: Spatial Features Extraction (Elevation & Land Cover)

This notebook extracts **spatial features**—elevation and land cover—for water quality prediction. These features help explain water quality drivers because:

- **Elevation** influences runoff, groundwater flow, and pollutant transport
- **Land cover** (e.g., forest vs. cropland vs. urban) affects erosion, nutrient runoff, and sedimentation

## Objectives
1. Load sample locations from Landsat training and validation data
2. Extract elevation from Copernicus DEM (30m resolution)
3. Extract land cover from ESA WorldCover (10m resolution)
4. Save spatial feature datasets for model integration

## Data Sources
- **Copernicus DEM GLO-30**: Global 30m elevation from TanDEM-X radar
- **ESA WorldCover**: Global 10m land cover classification (11 classes)

## Step 1: Load Dependencies and Data

We import the same stack used in other EY notebooks: `pystac_client` and `planetary_computer` for STAC API access, `rasterio` for reading cloud-optimized GeoTIFFs, and `pandas` for data handling.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import rasterio

import pystac_client
import planetary_computer as pc

from tqdm import tqdm
import os

In [2]:
# Load Landsat training and validation data (source of sample locations)
print("Loading sample locations from Landsat feature files...")
landsat_train = pd.read_csv("landsat_features_training.csv")
landsat_val = pd.read_csv("landsat_features_validation.csv")

print(f"Training samples: {len(landsat_train)}")
print(f"Validation samples: {len(landsat_val)}")
print(f"Columns: {list(landsat_train.columns)}")

# We need Latitude, Longitude, Sample Date for merging later
train_coords = landsat_train[["Latitude", "Longitude", "Sample Date"]].copy()
val_coords = landsat_val[["Latitude", "Longitude", "Sample Date"]].copy()

landsat_train.head()

Loading sample locations from Landsat feature files...
Training samples: 9319
Validation samples: 200
Columns: ['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']


,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683


## Step 2: Extract Elevation from Copernicus DEM

We use the **Copernicus DEM GLO-30** (30m resolution) from the Microsoft Planetary Computer. The DEM is derived from TanDEM-X radar. Elevation (meters above mean sea level) influences:

- Runoff and erosion
- Groundwater discharge

**Approach**: For each sample point, we search for the DEM tile containing that point, load the tile, and sample the elevation value at (lon, lat). We cache tiles to avoid repeated loads when many points fall in the same tile.

In [3]:
def extract_elevation_for_points(df: pd.DataFrame) -> pd.Series:
    """
    Extract elevation (meters) at each (Latitude, Longitude) using Copernicus DEM GLO-30.
    Returns a Series with elevation values aligned to the input DataFrame index.
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    # Study region bbox (Southern Africa)
    bbox = [14.97, -35.18, 32.79, -21.72]  # [min_lon, min_lat, max_lon, max_lat]
    search = catalog.search(
        collections=["cop-dem-glo-30"],
        bbox=bbox,
    )
    items = list(search.get_items())
    print(f"Found {len(items)} Copernicus DEM tiles covering the study region")

    # Map each item to its geometry for point-in-tile checks
    def get_bbox(item):
        coords = item.geometry["coordinates"][0]
        lons = [c[0] for c in coords]
        lats = [c[1] for c in coords]
        return min(lons), min(lats), max(lons), max(lats)

    item_bboxes = [(item, get_bbox(item)) for item in items]

    def find_tile_for_point(lon, lat):
        for item, (min_lon, min_lat, max_lon, max_lat) in item_bboxes:
            if min_lon <= lon <= max_lon and min_lat <= lat <= max_lat:
                return item
        return None

    # Asset key for elevation band
    elevation_asset_key = "data"

    # Cache: tile_id -> opened rasterio dataset (we'll sample and close per tile)
    tile_cache = {}

    elevations = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting elevation"):
        lat = row["Latitude"]
        lon = row["Longitude"]
        item = find_tile_for_point(lon, lat)
        if item is None:
            elevations.append(np.nan)
            continue

        asset = item.assets.get(elevation_asset_key) or item.assets.get("elevation")
        if asset is None:
            elevations.append(np.nan)
            continue

        tile_id = item.id
        try:
            if tile_id not in tile_cache:
                signed_asset = pc.sign(asset)
                tile_cache[tile_id] = rasterio.open(signed_asset.href)
            src = tile_cache[tile_id]
            # rasterio sample expects (x, y) = (lon, lat) in EPSG:4326
            vals = list(src.sample([(lon, lat)]))
            if vals:
                v = float(vals[0][0])
                # Copernicus uses -32768 as nodata
                elevations.append(np.nan if v <= -32768 else v)
            else:
                elevations.append(np.nan)
        except Exception as e:
            elevations.append(np.nan)

    # Close cached datasets
    for ds in tile_cache.values():
        ds.close()

    return pd.Series(elevations, index=df.index)

In [4]:
# Extract elevation for training and validation
print("Extracting elevation for training data...")
train_elev = extract_elevation_for_points(train_coords)
train_coords["elevation"] = train_elev

print("\nExtracting elevation for validation data...")
val_elev = extract_elevation_for_points(val_coords)
val_coords["elevation"] = val_elev

print(f"\nTraining elevation summary (m): {train_coords['elevation'].describe()}")
print(f"Validation elevation summary (m): {val_coords['elevation'].describe()}")

Extracting elevation for training data...
Found 226 Copernicus DEM tiles covering the study region


Extracting elevation: 100%|██████████| 9319/9319 [01:01<00:00, 150.77it/s] 



Extracting elevation for validation data...
Found 226 Copernicus DEM tiles covering the study region


Extracting elevation: 100%|██████████| 200/200 [00:08<00:00, 22.55it/s]


Training elevation summary (m): count    9319.000000
mean      909.345318
std       515.518178
min         2.463319
25%       369.233032
50%      1082.105591
75%      1315.000000
max      1590.156738
Name: elevation, dtype: float64
Validation elevation summary (m): count    200.000000
mean     379.425071
std      334.774593
min        2.000000
25%      129.090591
50%      192.466599
75%      773.500000
max      979.294922
Name: elevation, dtype: float64


## Step 3: Extract Land Cover from ESA WorldCover

**ESA WorldCover** provides global land cover at 10m resolution with 11 classes (e.g., Tree cover, Cropland, Built-up, Permanent water bodies). Land cover strongly influences water quality through:

- Urban/agricultural runoff (nutrients, sediments)
- Vegetation buffering and erosion control

**Note**: WorldCover 2020 is used as a proxy for land cover during 2011–2015. Land cover changes slowly, so this provides useful spatial context.

**Class codes**: 10=Tree cover, 20=Shrubland, 30=Grassland, 40=Cropland, 50=Built-up, 60=Bare, 70=Snow/ice, 80=Water bodies, 90=Herbaceous wetland, 95=Mangroves, 100=Moss/lichen

In [5]:
def extract_land_cover_for_points(df: pd.DataFrame) -> pd.Series:
    """
    Extract land cover class at each (Latitude, Longitude) using ESA WorldCover 2020.
    Returns a Series with land cover codes aligned to the input DataFrame index.
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    bbox = [14.97, -35.18, 32.79, -21.72]
    search = catalog.search(
        collections=["esa-worldcover"],
        bbox=bbox,
    )
    items = list(search.get_items())
    print(f"Found {len(items)} ESA WorldCover tiles covering the study region")

    def get_bbox(item):
        coords = item.geometry["coordinates"][0]
        lons = [c[0] for c in coords]
        lats = [c[1] for c in coords]
        return min(lons), min(lats), max(lons), max(lats)

    item_bboxes = [(item, get_bbox(item)) for item in items]

    def find_tile_for_point(lon, lat):
        for item, (min_lon, min_lat, max_lon, max_lat) in item_bboxes:
            if min_lon <= lon <= max_lon and min_lat <= lat <= max_lat:
                return item
        return None

    asset_key = "map"
    tile_cache = {}

    land_cover = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting land cover"):
        lat = row["Latitude"]
        lon = row["Longitude"]
        item = find_tile_for_point(lon, lat)
        if item is None:
            land_cover.append(np.nan)
            continue

        asset = item.assets.get(asset_key)
        if asset is None:
            land_cover.append(np.nan)
            continue

        tile_id = item.id
        try:
            if tile_id not in tile_cache:
                signed_asset = pc.sign(asset)
                tile_cache[tile_id] = rasterio.open(signed_asset.href)
            src = tile_cache[tile_id]
            vals = list(src.sample([(lon, lat)]))
            if vals:
                v = int(vals[0][0])
                land_cover.append(v)
            else:
                land_cover.append(np.nan)
        except Exception:
            land_cover.append(np.nan)

    for ds in tile_cache.values():
        ds.close()

    return pd.Series(land_cover, index=df.index)

In [6]:
# Extract land cover for training and validation
print("Extracting land cover for training data...")
train_lc = extract_land_cover_for_points(train_coords)
train_coords["land_cover"] = train_lc

print("\nExtracting land cover for validation data...")
val_lc = extract_land_cover_for_points(val_coords)
val_coords["land_cover"] = val_lc

print("\nLand cover value counts (training):")
print(train_coords["land_cover"].value_counts(dropna=False).sort_index())

Extracting land cover for training data...
Found 62 ESA WorldCover tiles covering the study region


Extracting land cover: 100%|██████████| 9319/9319 [00:34<00:00, 271.85it/s] 



Extracting land cover for validation data...
Found 62 ESA WorldCover tiles covering the study region


Extracting land cover: 100%|██████████| 200/200 [00:04<00:00, 44.18it/s] 


Land cover value counts (training):
land_cover
10    2869
20    1075
30    2885
40     204
50     357
60     352
80    1577
Name: count, dtype: int64


## Step 4: Save Spatial Feature Datasets

We save CSVs with `Latitude`, `Longitude`, `Sample Date`, `elevation`, and `land_cover`. These can be merged with Landsat and TerraClimate features in Phase 2, Step 6.

In [7]:
out_train = "spatial_features_training.csv"
out_val = "spatial_features_validation.csv"

train_coords.to_csv(out_train, index=False)
val_coords.to_csv(out_val, index=False)

print(f"Saved {out_train} ({len(train_coords)} rows)")
print(f"Saved {out_val} ({len(val_coords)} rows)")
print("\nPreview:")
train_coords.head(10)

Saved spatial_features_training.csv (9319 rows)
Saved spatial_features_validation.csv (200 rows)

Preview:


,Latitude,Longitude,Sample Date,elevation,land_cover
0,-28.760833,17.730278,02-01-2011,163.000000,80
1,-26.861111,28.884722,03-01-2011,1520.538940,30
2,-26.450000,28.085833,03-01-2011,1470.731201,30
3,-27.671111,27.236944,03-01-2011,1338.717163,30
4,-27.356667,27.286389,03-01-2011,1357.407837,30
5,-27.010111,26.698083,04-01-2011,1281.680786,10
6,-25.127778,27.628889,04-01-2011,951.590576,20
7,-25.206390,27.558000,04-01-2011,950.862854,30
8,-24.695140,27.409060,04-01-2011,912.183167,80
9,-26.984722,26.632278,04-01-2011,1277.647705,30


## Next Steps

- Phase 2, Step 5: Create temporal features
- Phase 2, Step 6: Combine all features (Landsat, TerraClimate, spatial, temporal)